# モデルのSave/Load

In [ ]:
# chainer
model_path = 'model.hdf5'
chainer.serializers.save_hdf5(model_path, model)
chainer.serializers.load_hdf5(model_path, model)

In [ ]:
# pytorch
model_path = 'model.pth'
torch.save(model.state_dict(), model_path) # for gpu
torch.save(model.to('cpu').state_dict(), model_path) # for cpu
model.load_state_dict(torch.load(model_path))

In [ ]:
# tensorflow v1
model_path = 'model'
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.save(sess, model_path)
    saver.restore(sess, model_path)

In [ ]:
# tensorflow v2
model_path = 'model'
tf.saved_model.save(model, model_path)
model = tf.saved_model.load(model_path)

# この他にもtf.keras.models.save_modelやmodel.save_weights()やmodel.to_json()を使った方法がある

# カスタムデータセット

In [ ]:
# chainer
class CustomDataset(chainer.dataset.DatasetMixin):
    def __init__(self, images_path, image_size, labels_path, dtype=np.float32):
        self._images_path = images_path
        self._image_size = image_size
        self._labels_path = labels_path
        self._dtype = dtype

        self._images = []
        self._labels = []
        images = os.listdir(self._images_path)
        labels = [0] * len(images)
        # 以下のようにファイルから読み込むパターンもある
        #for line in open(self._labels_path):
        #  self._labels.append(int(line.strip()))

        for image, label in zip(images, labels):
            self._images.append(image)
            self._labels.append(label)

    def __len__(self):
        # データセットの数を返す
        return len(self._images_path)

    def get_example(self, index):
        # データセットのインデックスを受け取ってデータを返す
        image = self._images[index]
        label = self._labels[index]
        with open(image, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
        image = image.resize(self._image_size)
        image = np.asarray(image, dtype=self._dtype)
        image = image.transpose(2, 0, 1) # PILのImageは(height, width, channel)なのでChainerの形式に変換
        return image, np.array(label)

custom_dataset = CustomDataset(images_path, (64, 64), labels_path)
train_iter = iterators.SerialIterator(dataset=custom_dataset, batch_size=64, shuffle=True)

In [ ]:
# pytorch
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.images = []
        self.labels = []
        images_path = "images_path"
        images = os.listdir(images_path)
        labels = [0] * len(images)
        # 以下のようにファイルから読み込むパターンもある
        #for line in open(os.path.join(root_path, name + '.txt')):
        #  self.labels.append(int(line.strip()))

        for image, label in zip(images, labels):
            self.images.append(image)
            self.labels.append(label)

    def __getitem__(self, index):
        # データセットのインデックスを受け取ってデータを返す
        image = self.images[index]
        label = self.labels[index]
        with open(image, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, np.array(label)

    def daya(self):
        # データセットの数を返す
        return len(self.images)

custom_dataset = CustomDataset(transform=torchvision.transforms.ToTensor()))
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset, batch_size=64, shuffle=True)

In [ ]:
# tensorflow v1

# 基本的にデータセットはNHWCの4階テンソルNumpy配列で与えればよい
class CustomDataset:
    def __init__(self, images_path, image_size, labels_path, dtype=np.float32):
        self._images_path = images_path
        self._image_size = image_size
        self._labels_path = labels_path
        self._dtype = dtype
        self._index_in_epoch = 0
        self._epochs_completed = 0

        self._images = []
        self._labels = []
        images = os.listdir(self._images_path)
        labels = [0] * len(images)
        # 以下のようにファイルから読み込むパターンもある
        #for line in open(self._labels_path):
        #  self._labels.append(int(line.strip()))

        for image, label in zip(images, labels):
            self._images.append(image)
            self._labels.append(label)

    def next_batch(self, batch_size, shuffle=False):
        # バッチサーズの分だけデータを返す
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self.datanum():
            # 1 epoch終わった後の処理
            self._epochs_completed += 1
            perm = np.arange(self.datanum())
            if shuffle:
                np.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            start = 0
            self._index_in_epoch = batch_size
        end = self._index_in_epoch
        
        image = self._images[start:end]
        label = self._labels[start:end]
        image_arr = []
        for image_ele in image:
            with open(image_ele, 'rb') as f:
                image_ele = Image.open(f)
                image_ele = image_ele.convert('RGB')
            image_ele = image_ele.resize(self._image_size)
            image_arr.append(image_ele)
        image = np.asarray(image_arr, dtype=self._dtype)
        return image, np.array(label)

    def datanum(self):
        # データセットの数を返す
        return len(self._images_path)

custom_dataset = CustomDataset(images_path, (64, 64), labels_path)
with tf.Session(config=config) as sess:
    # ・・・
    train_images, train_labels = custom_dataset.next_batch(batch_size=64, shuffle=True)
    # ・・・

In [ ]:
# tensorflow v2

def load_and_preprocess_image(path):
  # pathの画像を読み込み正規化する関数
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [64, 64])
  image /= 255.0
  return image

all_images_path = os.listdir(images_path)
all_labels = [0] * len(all_images_path)
path_ds = tf.data.Dataset.from_tensor_slices(all_images_path)
image_ds = path_ds.map(load_and_preprocess_image) # 前処理を適用
label_ds = tf.data.Dataset.from_tensor_slices(all_label)
custom_dataset = tf.data.Dataset.zip((image_ds, label_ds))
# 前処理が無ければ最初から画像とラベルを以下のように与えても良い
# custom_dataset = tf.data.Dataset.from_tensor_slices((all_images, all_labels)).batch(batch_size=64)
custom_dataset = custom_dataset.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_images_path)))
custom_dataset = custom_dataset.batch(64).prefetch(tf.data.experimental.AUTOTUNE)
train_images, train_labels = next(iter(custom_dataset))

# datasetをTFRecordに書きこんで読み込みを早くすることも可能
# https://www.tensorflow.org/tutorials/load_data/images?hl=ja

# tensorflow v2

In [ ]:
# MLP
# https://www.tensorflow.org/tutorials/quickstart/advanced?hl=ja
import tensorflow as tf

# 1.データセットの準備部分
batch_size = 50         # ミニバッチのサイズ
mnist = tf.keras.datasets.mnist

(train_images, train_labels),(test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

train_ds = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)).shuffle(10000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

# 2.モデルの作成部分
learning_rate = 0.001   # 学習率
training_epochs = 5     # エポック数

class MLPModel(tf.keras.Model):

    def __init__(self):
        super(MLPModel, self).__init__()
        self.flat1 = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        h = self.flat1(inputs)
        h = self.dense1(h)
        return self.dense2(h)

model = MLPModel()

# 3.実行する部分
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() # CategoricalCrossentropyと違い、ラベルのOne-hot化不要
optimizer = tf.keras.optimizers.Adam(learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss') # 損失の記録用
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') # 精度の記録用
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 訓練用
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# テスト用
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# 学習開始
for epoch in range(training_epochs):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  # 次のエポック用にメトリクスをリセット
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

In [ ]:
# CNN
import tensorflow as tf

# 1.データセットの準備部分
batch_size = 50         # ミニバッチのサイズ
mnist = tf.keras.datasets.mnist

(train_images, train_labels),(test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images = train_images[..., tf.newaxis] # NHWC (Number, height, width, channel)の形にするため次元追加
test_images = test_images[..., tf.newaxis]   # .reshapeを使っても良い

train_ds = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)).shuffle(10000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

# データ拡張を加える場合は以下のように書くこともできる
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=90)
# train_ds = datagen.flow(train_images, train_labels, batch_size, shuffle=True)
# test_ds = datagen.flow(test_images, test_labels, batch_size, shuffle=False)

# 2.モデルの作成部分
learning_rate = 0.001   # 学習率
training_epochs = 5     # エポック数
dropout = 0.5           # 非ドロップアウト率

class CNNModel(tf.keras.Model):

    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)) # input_shapeは無くても良い
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same') # tf.keras.layers.MaxPool2Dとも書ける
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='same')
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.flat1 = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        h = self.conv1(inputs)
        h = self.pool1(h)
        h = self.conv2(h)
        h = self.pool2(h)
        h = self.drop1(h)
        h = self.flat1(h)
        h = self.dense1(h)
        return self.dense2(h)

model = CNNModel()

# 3.実行する部分
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() # CategoricalCrossentropyと違い、ラベルのOne-hot化不要
optimizer = tf.keras.optimizers.Adam(learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss') # 損失の記録用
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') # 精度の記録用
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 訓練用
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# テスト用
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# 学習開始
for epoch in range(training_epochs):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  # 次のエポック用にメトリクスをリセット
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.10999956727027893, Accuracy: 96.58833312988281, Test Loss: 0.04337692633271217, Test Accuracy: 98.62999725341797
Epoch 2, Loss: 0.0377047173678875, Accuracy: 98.77999877929688, Test Loss: 0.04609948769211769, Test Accuracy: 98.54999542236328
Epoch 3, Loss: 0.025945182889699936, Accuracy: 99.13666534423828, Test Loss: 0.0295694712549448, Test Accuracy: 99.02999877929688
Epoch 4, Loss: 0.016230307519435883, Accuracy: 99.46333312988281, Test Loss: 0.03314388170838356, Test Accuracy: 98.90999603271484
Epoch 5, Loss: 0.014359348453581333, Accuracy: 99.53333282470703, Test Loss: 0.03502795472741127, Test Accuracy: 98.87999725341797


In [ ]:
# RNN
import tensorflow as tf

# 1.データセットの準備部分
batch_size = 50         # ミニバッチのサイズ
mnist = tf.keras.datasets.mnist

(train_images, train_labels),(test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

train_ds = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)).shuffle(10000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

# 2.モデルの作成部分
learning_rate = 0.001   # 学習率
training_epochs = 5     # エポック数

n_in = 28    # 1時刻当たりのデータ数
n_time = 28  # 時系列の数
n_unit = 128 # 中間層のセルの数

class RNNModel(tf.keras.Model):

    def __init__(self):
        super(RNNModel, self).__init__()
        # 入力の形を明示するためにtf.keras.layers.InputLayer(input_shape=(28,28))を最初に入れても良い
        stacked_cells = [tf.keras.layers.LSTMCell(n_unit) for _ in range(3)]
        self.lstm1 = tf.keras.layers.RNN(stacked_cells, return_sequences=True)
        self.dense1 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        h = self.lstm1(inputs)
        last_output = h[:,-1,:]
        h = self.dense1(last_output)
        return h

# 以下のようにも書ける
#class RNNModel(tf.keras.Model):
#
#    def __init__(self):
#        super(RNNModel, self).__init__()
#        self.lstm1 = tf.keras.layers.LSTM(n_unit, return_sequences=True)
#        self.lstm2 = tf.keras.layers.LSTM(n_unit, return_sequences=True)
#        self.lstm3 = tf.keras.layers.LSTM(n_unit, return_sequences=False)
#        self.dense1 = tf.keras.layers.Dense(10, activation='softmax')
#
#    def call(self, inputs):
#        h = self.lstm1(inputs)
#        h = self.lstm2(h)
#        h = self.lstm3(h)
#        h = self.dense1(h)
#        return h

model = RNNModel()

# 3.実行する部分
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() # CategoricalCrossentropyと違い、ラベルのOne-hot化不要
optimizer = tf.keras.optimizers.Adam(learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss') # 損失の記録用
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') # 精度の記録用
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 訓練用
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# テスト用
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# 学習開始
for epoch in range(training_epochs):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

  # 次のエポック用にメトリクスをリセット
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()